In [ ]:
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader,Dataset
import torch.optim as optim
import torch.nn.functional as F 
import os
import numpy as np

In [ ]:
class HandGestureDataset(Dataset):
    def __init__(self,filepath):
        xy=np.loadtxt(filepath,delimiter=',',dtype=np.float32)
        self.len=xy.shape[0]
        self.x_data=torch.from_numpy(xy[:,:-1])
        self.y_data=torch.from_numpy(xy[:,[-1]]).long()

    def __getitem__(self, index):
        return self.x_data[index],self.y_data[index]

    def __len__(self):
        return self.len

Training_set=HandGestureDataset('data\Training_set\Training_set_Label.csv')
train_loader=DataLoader(dataset=Training_set,shuffle=1,num_workers=0)

Test_set=HandGestureDataset('data\Test_set\Test_set_Label.csv')
test_loader=DataLoader(dataset=Test_set,batch_size=32,shuffle=1,num_workers=0)

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.l1=torch.nn.Linear(63,21)
        self.l2=torch.nn.Linear(21,5)
        self.l3=torch.nn.Linear(5,3)
        self.sigmoid=torch.nn.Sigmoid()

    def forward(self,x):
        x=self.sigmoid(self.l1(x))
        x=self.sigmoid(self.l2(x))
        return self.l3(x)


model=Net()

In [ ]:
criterion=torch.nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(),lr=0.005)

In [ ]:
def train(epoch):
    running_loss=0.0
    for batch_idx, data in enumerate(train_loader,0):
        inputs,target=data
        target=target.squeeze(0)

        optimizer.zero_grad()

        outputs=model(inputs)
        loss=criterion(outputs,target)
        loss.backward()
        optimizer.step()

        running_loss+=loss.item()

        if batch_idx % 40==39:
            print('[%d, %5d] loss: %.3f'%(epoch+1,batch_idx+1,running_loss))
            running_loss=0
def test():
    correct=0
    total=0
    with torch.no_grad():
        for data in test_loader:
            inputs,labels=data
            
            outputs=model(inputs)
            _,predicted=torch.max(outputs.data,dim=1)
            total+=labels.size(0)
            correct+=(predicted==labels.squeeze()).sum().item()
    
    print('Accuracy on test set: %d %%'%(100*correct/total))
    

In [ ]:
if __name__=='__main__':
    for epoch in range(350):
        train(epoch)
        
        test()
        

In [ ]:
torch.save(model,f='model.pkl') 